# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
'''
picking a test file
'''
whiteCarLane       = 'test_images/whiteCarLaneSwitch.jpg'    #passed
solidYellowLeft    = 'test_images/solidYellowLeft.jpg'       #passed
solidYellowCurve2  = 'test_images/solidYellowCurve2.jpg'     #passed
solidYellowCurve   = 'test_images/solidYellowCurve.jpg'      #passed
solidWhiteRight    = 'test_images/solidWhiteRight.jpg'       #passed
solidWhiteCurve    = 'test_images/solidWhiteCurve.jpg'       #passed


filename = solidWhiteCurve
testImage = mpimg.imread(filename)
print('This image is: ',type(testImage), 
         'with dimensions:', testImage.shape)
plt.imshow(testImage)

In [ ]:
"""
Color Selection:
Goal: Keep only yellow and white in the image (lane colors)
"""

def hk_color_select(img):
    # Grab the x and y size and make a copy of the image
    ysize = img.shape[0]
    xsize = img.shape[1]
    color_select = np.copy(img)
    
    # Define our color selection criteria
    red_threshold = 180
    green_threshold = 150
    blue_threshold = 0
    # max values of rgb allowed
    rgb_threshold = [red_threshold, green_threshold, blue_threshold]
    
    # Use a "bitwise OR" to identify pixels below the threshold
    thresholds = (img[:,:,0] < rgb_threshold[0]) \
                  | (img[:,:,1] < rgb_threshold[1]) \
                  | (img[:,:,2] < rgb_threshold[2])
    color_select[thresholds] = [0,0,0]
    
    return color_select
 

'''
run it...
'''    
testImage_cs = hk_color_select(testImage)


# Display the image                 
plt.imshow(testImage_cs)
    

In [ ]:
"""
Region of Interest:
Goal: Only keep marker that are in the center
"""

def hk_region_ofInterest(img):
    height = img.shape[0]
    width  = img.shape[1]
    
    height_mult  = 0.60
    width_mult_R = 0.54
    width_mult_L = 0.44
    
    # [horiz, vert]
    top_left     = [width*width_mult_L, height*height_mult]
    top_right    = [width*width_mult_R, height*height_mult]
    bottom_right = [width, height]
    bottom_left  = [0, height]
    
    poly = np.array([top_left, top_right, bottom_right, bottom_left], np.int32)
    return region_of_interest(img, [poly])

'''
run it...
'''    
testImage_roi = hk_region_ofInterest(testImage_cs)


# Display the image                 
plt.imshow(testImage_roi)


In [ ]:
"""
Grayscalling, Gaussian Smoothing & Canny Edge Detection:
Goal: Setup for Hough Transform
"""

def hk_canny_edgeDetect(img):
    #grayscale conversion
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Define a kernel size for Gaussian smoothing / blurring
    kernel_size = 3
    blur_gray = gaussian_blur(gray, kernel_size)

    # Define parameters for Canny and run it
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)
        
    return edges

    
'''
run it...
'''    
testImage_ced = hk_canny_edgeDetect(testImage_roi)

# Display the image                 
plt.imshow(testImage_ced, cmap='gray')  


In [ ]:
"""
Drawing lines:
Goal: Find the lanes
"""

def hk_find_2lineGroups(lines):
    
    lines_pos = np.zeros((0,1,4+2), dtype=lines.dtype)
    lines_neg = np.zeros((0,1,4+2), dtype=lines.dtype)
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            
            slope = (y2-y1)/(x2-x1)
            y_int = y1 - slope*x1
            
            if slope>0.5 and slope<0.8:
                lines_pos = np.append(lines_pos, [[[x1, y1, x2, y2, slope, y_int]]], axis=0) 
            
            if slope>-0.8 and slope<-0.5:
                lines_neg = np.append(lines_neg, [[[x1, y1, x2, y2, slope, y_int]]], axis=0)
                           
    return lines_pos, lines_neg
 

def hk_find_avgSlopeInt(lines_withSlopeInt):
    m_sum = 0
    b_sum = 0
    num_elements = lines_withSlopeInt.shape[0]
    
    for line in lines_withSlopeInt:
        for x1,y1,x2,y2,m,b in line:
            m_sum += m
            b_sum += b
     
    is_valid = False
    m_avg = 0
    b_avg = 0
    if num_elements>0:
        m_avg = m_sum/num_elements
        b_avg = b_sum/num_elements
        is_valid = True
    
    return m_avg, b_avg, is_valid


def hk_create_line(img_height, m_avg, b_avg):
    # point on bottom of image
    y1 = img_height - 1
    x1 = int( (y1-b_avg)/m_avg )
    
    height_mult = 0.60
    y2 = int( img_height*height_mult )
    x2 = int( (y2-b_avg)/m_avg )
    
    # make line
    return np.array([[x1, y1, x2, y2]])

    

def hk_draw_lines(img, lines, color=[255, 0, 0], thickness=2): 
    if lines is not None:
        pos, neg = hk_find_2lineGroups(lines)
    
        new_lines = np.zeros((0,1,4), dtype=lines.dtype)  
    
        m_avg, b_avg, is_valid = hk_find_avgSlopeInt(pos)
        if is_valid==True:
            p_line = hk_create_line(img.shape[0], m_avg, b_avg)
            new_lines = np.append(new_lines, [p_line], axis=0)      
    
        m_avg, b_avg, is_valid = hk_find_avgSlopeInt(neg)
        if is_valid==True:
            n_line = hk_create_line(img.shape[0], m_avg, b_avg)
            new_lines = np.append(new_lines, [n_line], axis=0)
    
        draw_lines(img, new_lines, [255, 0, 0], 6)        
            
    

In [ ]:
"""
Hough Transform line detection:
Goal: Find the lanes
"""

def hk_hough_transform(img):
    ### Define the Hough transform parameters ###

    # distance resolution in pixels of Hough grid
    rho = 2
    
    # angular resolution in rad of Hough grid
    theta = np.pi/180
    
    # min num votes (intersections in Hough grid cell)
    # higher num favors longer lines
    threshold = 45
    
    # min number of pixels making up a line
    min_line_length = 80
    
    # max gap in pixels between connectable line segments
    # higher num allows multiple dashes to be connected into a single line
    max_line_gap = 100         
    
    # Make blank the same size as our image to draw lines on
    line_image = np.copy(img)*0

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)

    # Iterate over the output "lines" and draw lines on a blank image
    #draw_lines(line_image, lines, color=[255, 0, 0], thickness=2)
    hk_draw_lines(line_image, lines, color=[255, 0, 0], thickness=2)
    
    return line_image
    

def hk_create_weightedImg(img, edges):
    # create dark image
    rows   = edges.shape[0]
    cols   = edges.shape[1]
    dType  = edges.dtype
    zero_grayscale = np.zeros(rows*cols, dtype=dType).reshape(rows, cols)

    # Create a "color" binary image to combine with line image - purple
    color_edges = np.dstack((edges, zero_grayscale, edges))

    # Draw the lines on the image
    return weighted_img(color_edges, img)

'''
run it...
'''
testImage_cEdges = hk_hough_transform(testImage_ced)
combo = hk_create_weightedImg(testImage, testImage_cEdges)

# Display the image
plt.imshow(combo)
    

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
        
    frame_cs     = hk_color_select(image)    
    frame_roi    = hk_region_ofInterest(frame_cs)    
    frame_ced    = hk_canny_edgeDetect(frame_roi)    
    frame_cEdges = hk_hough_transform(frame_ced)
    
    result       = hk_create_weightedImg(image, frame_cEdges)
    return result

In [ ]:
'''
print out all the anotated stills
'''

# get list of stills
stills_list = os.listdir("test_images/")
for still in stills_list:
    
    # open the image
    filepath = "test_images/" + still
    input_still  = mpimg.imread(filepath)
    
    # process the image
    output_still = process_image(input_still)
    
    # convert to RGB
    output_still = cv2.cvtColor(output_still, cv2.COLOR_BGR2RGB)
    
    # create output filename
    pivot_pt = len(filepath)-1-3
    output_filename = filepath[:pivot_pt] + '_output' + filepath[pivot_pt:]
    
    # save the image
    cv2.imwrite(output_filename, output_still)


Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


Answer: How could you imagine making your algorithm better / more robust?
-----------------------------------------------------------------

One of the situations where I could make things more robust is in the combining the lines into 2 lines (left and right). I currently just average the slope and y-intercept. I could spend more effect to find the distribution and weight the numbers or elemenate the outliers (i.e. 'lines' largest and smallest slopes).

Also, below I list some places the algorithm may fail. These mostly involve parameters. I could try to make these variables adaptive, where I used CV to find features and calculate the parameters.


Answer: Where will your current algorithm be likely to fail?
-----------------------------------------------------

1. It will have problems if the lane dashes have a lot of space between them. That can be adjusted in 
   hk_hough_transform(img) with max_line_gap.
2. It will have problems if the lanes are very wide. That can be adjusted in hk_region_ofInterest() with bottom_right   
   and bottom_left.
3. I'll have problems in foreign countries if the the lane lines/dashes are not white or yellow. That can be asjusted  
   in hk_color_select() with red_threshold, green_threshold, and blue_threshold.
4. The algorithm picks yellow and white lines on dark pavement. When the pavement is light colored, it is very hard to   
   do Canny Edge detection accurately.   



## Submission

If you're satisfied with your video outputs it's time to submit!


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))